In [5]:
import numpy as np
import pandas as pd
from pulp import *


# 读取作物信息表，仅包含作物编号和作物名称两列
crop_info = pd.read_excel('作物信息（附件1表2）.xlsx', usecols=['作物编号', '作物名称'])

# 去除空格
crop_info.columns = [col.strip() for col in crop_info.columns]
crop_info['作物名称'] = crop_info['作物名称'].str.strip()

# 删除包含“羊肚菌”下方的无效数据
crop_info = crop_info.dropna(subset=['作物编号', '作物名称'])  # 删除空值行
crop_info = crop_info[crop_info['作物编号'].apply(lambda x: str(x).isdigit())]  # 仅保留作物编号为数字的行

# 建立作物名称到作物编号的映射关系
crop_mapping = dict(zip(crop_info['作物名称'], crop_info['作物编号']))
# print(crop_mapping)

merged_data = pd.read_excel('2023产量表.xlsx')
total_yield_by_crop = merged_data.groupby('作物名称')['总产量/斤'].sum().reset_index()

#print(total_yield_by_crop)


# 根据每块地的面积设置权重 S ，根据每种地的类型设置特定的种植矩阵
data = pd.read_excel('乡村现有耕地（附件1表1）.xlsx')
S = data['地块面积/亩']


# 以23年产量为预期销量
yield_mapping = total_yield_by_crop.set_index('作物名称')['总产量/斤'].to_dict()
Pre = [yield_mapping.get(crop_name, 0) for crop_name, _ in sorted(crop_mapping.items(), key=lambda x: x[1])]
# print(Pre)


data = pd.read_excel('作物收入汇总.xlsx')

# print(data[['作物编号', '是否豆类']].dropna().head())  # 检查前几行数据，确保没有缺失值
O = data['亩产量/斤']
N = data['作物']
P = data['中位单价收入/元']
C = data['种植成本/(元/亩)']



# z_2023:2023年种植情况
#          作物编号
#           1  2  3  4  5  6  7  8  9 10 11 12 13 14 15     A地
z_2023 = [[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # A1
           [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], # A2
           [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], # A3
           [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # A4
           [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # A5
           [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]],# A6
#           1  2  3  4  5  6  7  8  9 10 11 12 13 14 15     B地
          [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # B1
           [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # B2
           [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # B3
           [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # B4
           [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # B5
           [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], # B6
           [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # B7
           [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], # B8
           [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], # B9
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], # B10
           [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # B11
           [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], # B12
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], # B13
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]],# B14
#           1  2  3  4  5  6  7  8  9 10 11 12 13 14 15     C地
          [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], # C1
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], # C2
           [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # C3
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], # C4
           [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # C5
           [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],# C6
#          16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37     D地
          [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], # D1
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], # D2
           [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], # D3
           [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], # D4
           [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], # D5
           [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], # D6
           [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # D7
           [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],# D8
#          17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 38 39 40 41     E地
          [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], # E1
           [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], # E2
           [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], # E3
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], # E4
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], # E5
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], # E6
           [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], # E7
           [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], # E8
           [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], # E9
           [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], # E10
           [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], # E11
           [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], # E12
           [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], # E13
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], # E14
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1], # E15
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]],# E16
#          17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34
          [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # F1
           [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],  # F2
           [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],  # F3
           [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]  # F4
           ]

is_bean = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0]

# is_bean_X:地块X可种植的作物列表是否为豆类
is_bean_A = is_bean[:15] # is_bean[0]~is_bean[14]
is_bean_B = is_bean[:15]
is_bean_C = is_bean[:15]
is_bean_D = is_bean[15:37] # is_bean[15]~is_bean[36]

is_bean_E_1 = is_bean[16:34]
is_bean_E_2 = is_bean[37:41]
is_bean_E = np.concatenate([is_bean_E_1, is_bean_E_2])

is_bean_F = is_bean[16:34]
is_bean_F = is_bean_F + is_bean_F

# 2023年种植情况
planting_2023 = {
    'A': {
        'A1': [6], 'A2': [7], 'A3': [7], 'A4': [1], 'A5': [4], 'A6': [8]
    },
    'B': {
        'B1': [6], 'B2': [2], 'B3': [3], 'B4': [4], 'B5': [5], 'B6': [8],
        'B7': [6], 'B8': [8], 'B9': [9], 'B10': [10], 'B11': [1], 'B12': [7],
        'B13': [14], 'B14': [15]
    },
    'C': {
        'C1': [11], 'C2': [12], 'C3': [1], 'C4': [13], 'C5': [6], 'C6': [3]
    },
    'D': {
        'D1': [20, 36], 'D2': [28, 35], 'D3': [21, 35], 'D4': [22, 35],
        'D5': [17, 36], 'D6': [18, 37], 'D7': [16], 'D8': [16]
    },
    'E': {
        'E1': [18, 38], 'E2': [24, 38], 'E3': [25, 38], 'E4': [26, 39],
        'E5': [28, 39], 'E6': [27, 39], 'E7': [19, 40], 'E8': [19, 40],
        'E9': [18, 40], 'E10': [17, 41], 'E11': [17, 41], 'E12': [22, 41],
        'E13': [21, 41], 'E14': [29, 41], 'E15': [30, 27, 41], 'E16': [31, 41]
    },
    'F': {
        'F1': [32, 33, 24, 21], 'F2': [25, 26, 22, 29], 'F3': [17, 28, 30],
        'F4': [19, 34, 23]
    }
}


# S_X:地块X各小地块面积
S_A = S[0:6].values
S_B = S[6:20].values
S_C = S[20:26].values
S_D = S[26:34].values
S_E = S[34:50].values
S_F = S[50:54].values


# r_A:地块A能种的作物的利润
r_A = data['每亩利润'][:15].values # 平旱地
r_B = data['每亩利润'][15:30].values # 梯田
r_C = data['每亩利润'][30:45].values # 山坡地

r_D_1 = data['每亩利润'].iloc[45:64].values  # 第一个区段（45到63，包含 64 不包括）
r_D_2 = data['每亩利润'].iloc[82:85].values  # 第二个区段（83到84，包含 85 不包括）
r_D = np.concatenate([r_D_1, r_D_2]) #水浇地 = 水浇地 + 水浇地一 + 水浇地二

r_E_1 = data['每亩利润'].iloc[64:82].values
r_E_2 = data['每亩利润'].iloc[85:89].values
r_E =  np.concatenate([r_E_1, r_E_2]) #水浇地 = 水浇地 + 水浇地一 + 水浇地二 # 普通大棚 = 普通大棚一 + 普通大棚二

r_F = data['每亩利润'][89:125].values # 智慧大棚 = 智慧大棚一 + 智慧大棚二


# O_X：地块X作物产出
O_A = O[:15].values
O_B = O[15:30].values
O_C = O[30:45].values

O_D_1 = O.iloc[45:64].values
O_D_2 = O.iloc[82:85].values
# 拼接
O_D = np.concatenate([O_D_1, O_D_2])

O_E_1 = O.iloc[64:82].values
O_E_2 = O.iloc[85:89].values
O_E = np.concatenate([O_E_1, O_E_2])

O_F = O[89:125].values


# N_X:地块X可种植作物名称
N_A = N[:15].values
N_B = N[15:30].values
N_C = N[30:45].values

N_D_1 = N.iloc[45:64].values
N_D_2 = N.iloc[82:85].values
# 拼接
N_D = np.concatenate([N_D_1, N_D_2])

N_E_1 = N.iloc[64:82].values
N_E_2 = N.iloc[85:89].values
N_E = np.concatenate([N_E_1, N_E_2])

N_F = N[89:125].values


# P_A:地块A能种的作物的单价
P_A = P[:15].values # 平旱地
P_B = P[15:30].values # 梯田
P_C = P[30:45].values # 山坡地

P_D_1 = P.iloc[45:64].values  # 第一个区段（45到63，包含 64 不包括）
P_D_2 = P.iloc[82:85].values  # 第二个区段（83到84，包含 85 不包括）
P_D = np.concatenate([P_D_1, P_D_2]) #水浇地 = 水浇地 + 水浇地一 + 水浇地二

P_E_1 = P.iloc[64:82].values
P_E_2 = P.iloc[85:89].values
P_E =  np.concatenate([P_E_1, P_E_2]) #水浇地 = 水浇地 + 水浇地一 + 水浇地二 # 普通大棚 = 普通大棚一 + 普通大棚二

P_F = P[89:125].values # 智慧大棚 = 智慧大棚一 + 智慧大棚二


# C_A:地块A能种的作物的每亩成本
C_A = C[:15].values # 平旱地
C_B = C[15:30].values # 梯田
C_C = C[30:45].values # 山坡地

C_D_1 = C.iloc[45:64].values  # 第一个区段（45到63，包含 64 不包括）
C_D_2 = C.iloc[82:85].values  # 第二个区段（83到84，包含 85 不包括）
C_D = np.concatenate([C_D_1, C_D_2]) #水浇地 = 水浇地 + 水浇地一 + 水浇地二

C_E_1 = C.iloc[64:82].values
C_E_2 = C.iloc[85:89].values
C_E =  np.concatenate([C_E_1, C_E_2]) #水浇地 = 水浇地 + 水浇地一 + 水浇地二 # 普通大棚 = 普通大棚一 + 普通大棚二

C_F = C[89:125].values # 智慧大棚 = 智慧大棚一 + 智慧大棚二


Pre_A = Pre[:15]
Pre_B = Pre[:15]
Pre_C = Pre[:15]
Pre_D = Pre[15:36]

# 使用 np.concatenate 合并两个部分的数据
Pre_E = np.concatenate((Pre[16:34], Pre[37:41]))
Pre_F = np.concatenate((Pre[16:34], Pre[16:34]))

# print(Pre_A)
# print(Pre_B)
# print(Pre_C)
# print(Pre_D)
# print(Pre_E)
# print(Pre_F)


problem = LpProblem("Maximize_Profit", LpMaximize)


# 定义变量：作物种植比例w_X（连续变量）和作物种植决策z_X_i（0-1变量）
# w_A[year][i][j] 表示第 year 年地块 A 的第 i 块地对于第 j 种作物的种植比例，而 z_A[year][i][j] 则是其第 year 年地块 A 的第 i 块地是否种植第 j 种作物的决策变量
# w'_A 是 0 到 10 之间的整数,从而控制w_X范围[0, 1],且为0.1倍数
# 0 到 10 之间的整数变量（用于定义 w_X_prime）
# 定义2024-2030年每年的变量
years = range(2024, 2031)

def create_variables(prefix, S, N, years):
    w_prime = {year: [[LpVariable(f"{prefix}_prime_{year}_{i}_{j}", 0, 10, cat='Integer') for j in range(len(N))] for i in range(len(S))] for year in years}
    w = {year: [[LpVariable(f"{prefix}_{year}_{i}_{j}", 0, 1) for j in range(len(N))] for i in range(len(S))] for year in years}
    z = {year: [[LpVariable(f"z_{prefix}_{year}_{i}_{j}", 0, 1, cat="Binary") for j in range(len(N))] for i in range(len(S))] for year in years}
    return w_prime, w, z


w_A_prime, w_A, z_A = create_variables("A", S_A, N_A, years)
w_B_prime, w_B, z_B = create_variables("B", S_B, N_B, years)
w_C_prime, w_C, z_C = create_variables("C", S_C, N_C, years)
w_D_prime, w_D, z_D = create_variables("D", S_D, N_D, years)
w_E_prime, w_E, z_E = create_variables("E", S_E, N_E, years)
w_F_prime, w_F, z_F = create_variables("F", S_F, N_F, years)

# 添加 w_X_prime 与 w_X 之间的关系约束
def add_w_constraints(problem, w_prime, w, S, N, years):
    for year in years:
        for i in range(len(S)):
            for j in range(len(N)):
                problem += w[year][i][j] == w_prime[year][i][j] * 0.1        # 约束w为0.1的倍数


add_w_constraints(problem, w_A_prime, w_A, S_A, N_A, years)
add_w_constraints(problem, w_B_prime, w_B, S_B, N_B, years)
add_w_constraints(problem, w_C_prime, w_C, S_C, N_C, years)
add_w_constraints(problem, w_D_prime, w_D, S_D, N_D, years)
add_w_constraints(problem, w_E_prime, w_E, S_E, N_E, years)
add_w_constraints(problem, w_F_prime, w_F, S_F, N_F, years)


S_list = [S_A, S_B, S_C, S_D, S_E, S_F]
w_list = [w_A, w_B, w_C, w_D, w_E, w_F]
P_list = [P_A, P_B, P_C, P_D, P_E, P_F]
O_list = [O_A, O_B, O_C, O_D, O_E, O_F]
C_list = [C_A, C_B, C_C, C_D, C_E, C_F]
Pre_list = [Pre_A, Pre_B, Pre_C, Pre_D, Pre_E, Pre_F]
N_list = [N_A, N_B, N_C, N_D, N_E, N_F]

print("done")

done


In [9]:
crop_list = crop_info['作物名称']


# 总产量表
total_output = {year: {} for year in years}  
# 超额产量表
excess_output = {year: {} for year in years}


for year in years:
    if year not in total_output:
        total_output[year] = {}
        excess_output[year] = {}
    for crop in crop_list:
        crop_id = crop_mapping.get(crop)
        total_output[year][crop_id] = 0
        excess_output[year][crop_id] = 0
        for N_X, w_X, O_X, S_X, Pre_X in N_list, w_list, O_list, S_list, Pre_list:
            if crop in N_X:  # 如果X地可种作物crop
                crop_index_in_N = N_X.index(crop)
                
                total_output[year][crop_id] += lpSum([w_X[year][i][crop_index_in_N] * O_X[i][crop_index_in_N] * S_N[i] for i in range(len(S_N))])

                 # 创建一个 LP 变量来表示超额产量
                excess_var = LpVariable(f"excess_{year}_{crop_mapping.get(crop_id)}", lowBound=0, cat='Continuous')
                
                # 添加限制条件
                problem += excess_var >= total_output[year][crop_id] - Pre_X
                problem += excess_var >= 0
                excess_output[year][crop_id] += excess_var
        

{'黄豆': 1, '黑豆': 2, '红豆': 3, '绿豆': 4, '爬豆': 5, '小麦': 6, '玉米': 7, '谷子': 8, '高粱': 9, '黍子': 10, '荞麦': 11, '南瓜': 12, '红薯': 13, '莜麦': 14, '大麦': 15, '水稻': 16, '豇豆': 17, '刀豆': 18, '芸豆': 19, '土豆': 20, '西红柿': 21, '茄子': 22, '菠菜': 23, '青椒': 24, '菜花': 25, '包菜': 26, '油麦菜': 27, '小青菜': 28, '黄瓜': 29, '生菜': 30, '辣椒': 31, '空心菜': 32, '黄心菜': 33, '芹菜': 34, '大白菜': 35, '白萝卜': 36, '红萝卜': 37, '榆黄菇': 38, '香菇': 39, '白灵菇': 40, '羊肚菌': 41}
['黄豆' '黑豆' '红豆' '绿豆' '爬豆' '小麦' '玉米' '谷子' '高粱' '黍子' '荞麦' '南瓜' '红薯' '莜麦'
 '大麦']


ValueError: too many values to unpack (expected 4)